# Transformations
Point cloud processing generally uses transformation (translation, rotation, etc.) since point cloud data is a set of coordinates. In this section, we introduce transformation for a point cloud. Contents in this section are as follows:

- Translation, Rotation, Scaling
- Transformation matrix


In [1]:
%load_ext autoreload
%autoreload 2

## Translation, Rotation, Scaling
In this subsection, we introduce simple calculations of transformation. This subsection deals with translation, rotation, and scaling used in transformation. Examples of a transformation are as follows.

In [1]:
# for translation, rotation scaling
import numpy as np
from tutlibs.transformation import Transformation as tr

# for description
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.utils import single_color
from tutlibs.io import Points as io
import inspect

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
# load a point cloud.
coords, _, _ = io.read('../data/bunny_pc.ply')

# translation
translated_coords = tr.translation(coords, np.array([0.5, 0.8, 0.2]))

# rotation
rotated_coords = tr.rotation(coords, "x", np.pi/2)

# scaling
scaled_coords = tr.scaling(coords, np.array([0.5, 0.5, 0.5]))

# visualization
obj_points = jv.point(coords, single_color('#000000', len(coords)))
obj_translated_points = jv.point(translated_coords, single_color('#ff0000', len(translated_coords)))
obj_rotated_points = jv.point(rotated_coords, single_color('#00ff00', len(rotated_coords)))
obj_scaled_points = jv.point(scaled_coords, single_color('#0000ff', len(scaled_coords)))
jv.display([obj_translated_points, obj_rotated_points, obj_scaled_points, obj_points])

Output()

<!-- 上記の出力では、黒の点群がオリジナル、赤が平行移動、緑が回転、青がスケーリングとなっています。これらの変換は全て原点を中心として処理がなされているため、オブジェクトを中心として変換がなされていないことに注意してください。

次に、これらの処理について確認します。 -->

In the above output, the black point cloud is the original, red is translated, green is rotated, and blue is scaled. Note that the center coordinates of these transformations are not an object centroid, but the origin. For example, the green point cloud is not rotating around the object centroid.

Next, let's check these processes.

In [7]:
print(inspect.getsource(tr.translation))
print(inspect.getsource(tr.rotation))
print(inspect.getsource(tr.scaling))

    @staticmethod
    def translation(xyz: np.ndarray, vector: np.ndarray) -> np.ndarray:
        """Shift xyz.
        Args:
            xyz: (N, 3)
            vector: (3)

        Return:
            translation xyz: (N, 3)
        """
        return xyz + vector[np.newaxis, :]

    @staticmethod
    def rotation(xyz: np.ndarray, axis: str, angle: float) -> np.ndarray:
        """Rotate xyz.

        Args:
            xyz: (N, 3)
            axis: x, y or z
            angle: radian (0 ~ 2pi)

        Return:
            rotated xyz: (N, 3)
        """
        if axis == "x":
            rotation_matrix = np.array(
                [
                    [1, 0, 0],
                    [0, np.cos(angle), -np.sin(angle)],
                    [0, np.sin(angle), np.cos(angle)],
                ]
            )
        elif axis == "y":
            rotation_matrix = np.array(
                [
                    [np.cos(angle), 0, np.sin(angle)],
                    [0, 1, 0],
            

The above transformations for each point are the same as the following equations.

- Translation
$$
\begin{equation}
\left[\begin{array}{ccc}
p_x' \\
p_y' \\
p_z'
\end{array}\right]
=
\left[\begin{array}{ccc}
p_x \\
p_y \\
p_z
\end{array}\right]
+
\left[\begin{array}{ccc}
t_x \\
t_y \\
t_z
\end{array}\right] 
\end{equation}
$$

- Rotation

$$
\left[\begin{array}{ccc}
p_x' \\
p_y' \\
p_z'
\end{array}\right]
=\left[\begin{array}{ccc}
r_1 & r_2 & r_3 \\
r_4 & r_5 & r_6 \\
r_7 & r_8 & r_9
\end{array}\right]
\left[\begin{array}{ccc}
p_x \\
p_y \\
p_z
\end{array}\right]
$$

- Scaling ($\odot$ is hadamard product)
$$
\begin{equation}
\left[\begin{array}{ccc}
p_x' \\
p_y' \\
p_z'
\end{array}\right]
=
\left[\begin{array}{ccc}
p_x \\
p_y \\
p_z
\end{array}\right]
\odot
\left[\begin{array}{ccc}
t_x \\
t_y \\
t_z
\end{array}\right] 
\end{equation}
$$

As above, translation and scaling are simple ways by adding or multiplying each element.

However, Rotation use dot products with the 3x3 rotation matrix. Elements in 3x3 rotation matrix depend on rotation axes.

## Transformation matrix
Except for the above (Translation, Rotation, Scaling), There are transformation methods with 4x4 transformation matrix. These methods compute the dot product for a point cloud and transformation matrix. This transformation matrix can have transformation information that includes translation vectors, scaling values, and rotation matrix. 

An example of a transformation is as follows.


In [1]:
# for transformation matrix
import numpy as np
from tutlibs.transformation import (
    TransformationMatrix as tm,
    Transformation as tr,
)

# for description
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.utils import single_color
from tutlibs.io import Points as io
import inspect


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
# load a point cloud.
coords, _, _ = io.read("../data/bunny_pc.ply")

angle = np.pi/3
rotation_matrix = np.array(
    [
        [1, 0, 0],
        [0, np.cos(angle), -np.sin(angle)],
        [0, np.sin(angle), np.cos(angle)],
    ]
)

transformation_matrix_1 = tm.from_translation(np.array([0.5, 0.8, 0.2]))
transformation_matrix_2 = tm.from_rotation(rotation_matrix)
transformation_matrix_3 = tm.from_scaling(np.array([0.5, 0.5, 0.5]))

print(f"transformation_matrix_1: \n {transformation_matrix_1} \n")
print(f"transformation_matrix_2: \n {transformation_matrix_2} \n")
print(f"transformation_matrix_3: \n {transformation_matrix_3} \n")

transformation_matrix = tm.composite(
    [transformation_matrix_1, transformation_matrix_2, transformation_matrix_3]
)

print(f"transformation matrix: \n {transformation_matrix} \n")

tranformed_coords = tm.transformation(coords, transformation_matrix)

# visualization
obj_points = jv.point(coords, single_color("#000000", len(coords)))
obj_transformed_points = jv.point(
    tranformed_coords, single_color("#ff0000", len(tranformed_coords))
)

jv.display([obj_points, obj_transformed_points])


transformation_matrix_1: 
 [[ 1.         0.         0.         0.       ]
 [ 0.         0.5       -0.8660254  0.       ]
 [ 0.         0.8660254  0.5        0.       ]
 [ 0.         0.         0.         1.       ]] 

transformation_matrix_2: 
 [[1.  0.  0.  0.5]
 [0.  1.  0.  0.8]
 [0.  0.  1.  0.2]
 [0.  0.  0.  1. ]] 

transformation_matrix_3: 
 [[0.5 0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  1. ]] 

transformation matrix: 
 [[ 0.5         0.          0.          0.5       ]
 [ 0.          0.25       -0.4330127   0.22679492]
 [ 0.          0.4330127   0.25        0.79282032]
 [ 0.          0.          0.          1.        ]] 



Output()

As shown above, we can integrate the transformation information into single transformation matrix by dot product.
Since the transformation matrix is a simple transformation representation, the transformation matrix is used in various cases such as datasets and iterative methods (ex: ICP).

Next, let's check the contents of the 4x4 transformation matrix. Relations between a 4x4 transformation matrix and others are as follows.

- convert a translation vector $\mathbf{T}$ to a 4x4 transformation matrix $\mathbf{TM}$:

$$
\mathbf{T}=\left[\begin{array}{ccc}
t_x \\
t_y \\
t_z
\end{array}\right] , 
\mathbf{TM}=\left[\begin{array}{ccc}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z \\
0 & 0 & 0 & 1
\end{array}\right]
$$

- convert a 3x3 rotation matrix $\mathbf{R}$ to a 4x4 transformation matrix $\mathbf{TM}$:

$$
\mathbf{R}=\left[\begin{array}{ccc}
r_1 & r_2 & r_3 \\
r_4 & r_5 & r_6 \\
r_7 & r_8 & r_9
\end{array}\right] , 
\mathbf{TM}=\left[\begin{array}{ccc}
r_1 & r_2 & r_3 & 0 \\
r_4 & r_5 & r_6 & 0 \\
r_7 & r_8 & r_9 & 0 \\
0 & 0 & 0 & 1
\end{array}\right]
$$

- convert a scaling matrix $\mathbf{S}$ to a 4x4 transformation matrix $\mathbf{TM}$:
$$
\mathbf{S}=\left[\begin{array}{ccc}
s_x \\
s_y \\
s_z
\end{array}\right] , 
\mathbf{TM}=\left[\begin{array}{ccc}
s_x & 0 & 0 & 0 \\
0 & s_y & 0 & 0 \\
0 & 0 & s_z & 0 \\
0 & 0 & 0 & 1
\end{array}\right]
$$
